<a href="https://colab.research.google.com/github/mmcuervo/ExamMLcategorias/blob/Experimento3/03_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook3 MODEL TRAINING: En este archivo entrenamos el model**

**Problema:** El equipo de atención al cliente recibe cerca de 1000 tickets por día, algunos son urgentes y otros no, necesitamos un sistema que clasifique los tickets por prioridad, alta, media y baja. Para que se atiendan los de prioridad alta primero.

**Pregunta que responde el modelo:** ¿Cuál es la prioridad de el ticket analizado?

**Pregunta que responde este notebook:** ¿Los datos de entrada son acordes con el dataset de entrenamiento?

**Acciones a realizar:**  

1.   Modifacar el dataSet de acuerdo a las reglas del EDA para que los datos de producción sean homogéneos a los datos de entrenamiento.
2.   Crear un informe log, sobre la calidad de los datos recibidos.

**CONTRATO DE DATOS:**

*Variables necesarias para analizar el modelo:*

 **Variables categóricas:** company_size,industry, customer_tier, product_area
** Variables numéricas (int64):** past_30d_tickets, past_90d_incidents,description_length, customers_affected,downtime_min
** Variables numéricas (float64):** error_rate_pct
 **Variables booleanas (int64):** payment_impact_flag, data_loss_flag, security_incident_flag, has_runbook  

**Tratamiento de data_type erroneo:**
Todos los casos: Clasificarlo como nulo (UKNOWN para texto , 0 para numeros) y tratarlos como nulos

***Reglas Datos nulos***

Tratamiento de datos nulos:

**Categorías nominales:** Se registra **UNKNOWN** y se clasifican como **other**
**Categorías numéricas:** Imputar con mediana
**Categoría boolena:** Remplazar nulo con 0

Siempre registrar un Log de los datos nulos para reporte de calidad de dataSet Interpretación de manejo de datos nulos que superen el valor de entrenamiento:** 0–5:%** ✅ aceptable. **5–10:% ** ⚠️ investigar. >10:%🚨 problema de calidad. Datos nulos en customer_sentment =1.8%, resto de las variables 0%

**Tratamiento de datos no previstos**
**Datos categóricos:** Si es texto se respeta el feature pero se clasifica como other, si es número se registra como **UNKNOWN** y se clasifica como other
Datos booleanos:Si es texto se decalara como nulo y se aplica la regla de dato nulo, si es número (ej: 3) se remplaza con cero

Siempre registrar un Log de los datos nulos para reporte de calidad de dataSet
Interpretación de manejo de datos nulos que superen el valor de entrenamiento**:0-5:**% ✅ aceptable.**5-10**:%  ⚠️ investigar. **>10**:% 🚨 problema de calidad. Datos nulos en customer_sentment =**1.8**%, resto de las variables **0**%**

**Subcategorias de datos categóricos:**
--- Columna: company_size ---
array(['Small', 'Medium', 'Large'], dtype=object)

--- Columna: industry ---
array(['media', 'healthcare', 'gaming', 'ecommerce', 'saas_b2b',
       'logistics', 'fintech'], dtype=object)

--- Columna: customer_tier ---
array(['Basic', 'Plus', 'Enterprise'], dtype=object)

--- Columna: product_area ---
array(['mobile', 'analytics', 'notifications', 'auth', 'billing',
       'data_pipeline'], dtype=object)

**Criterios de escalamiento**

**Datos numéricos:**
Aplicar Log-transform (log1p) + Z-score a:
downtime_min, customer_affected y error_rate_pct.
Eliminar columnas originales quedarse con las de sufijo _log
Escalar con Z-score a:
past_30d_tickets, past_90d_incidents, y description_length.

Datos categóricos y booleanos no se escalan
Reportar min y max  de los datos en un log para análisis de calidad de los datos

**Criterio de diseño de dataset final:**
Exportar el dataset final siguiendo los lineamientos de Tidy Data

**Criterios target:** Equilibrar priority_cat con Class-weight, no aplica hook los numeros si son jerárquicos

# **Ingesta del DataSet**

In [ ]:
# 1) Importar librerías básicas para EDA
import pandas as pd              # Manejo de datos en DataFrames
import numpy as np               # Utilidades numéricas (vectores, matrices, NaN)
import matplotlib.pyplot as plt  # Gráficas base
import seaborn as sns            # Gráficas estadísticas (opcional, solo EDA)
from pathlib import Path         # Manejo seguro de rutas de archivos


In [ ]:
#2 Llamamos al dataset y creamos una copia para trabajar en ella y no tocar el original
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = "/content/drive/MyDrive/AiLabData/Support_tickets.csv"
df = pd.read_csv(DATA_PATH, encoding="utf-8")
dc = df.copy()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#3) Verificar que el dataset esté completo y eliminar columnas sobrantes, Verificamos si priority_cat tiene datos válidos (1,2,3) o si nulos/inválidos >50% (eliminar priority_cat)/ priority_cat no existe (producción)

# Definir las columnas esperadas y sus tipos de datos basados en el contrato
expected_columns = {
    'company_size': 'object',
    'industry': 'object',
    'customer_tier': 'object',
    'past_30d_tickets': 'int64',
    'past_90d_incidents': 'int64',
    'product_area': 'object',
    'customers_affected': 'int64',
    'error_rate_pct': 'float64',
    'downtime_min': 'int64',
    'payment_impact_flag': 'int64',
    'data_loss_flag': 'int64',
    'security_incident_flag': 'int64',
    'has_runbook': 'int64',
    'description_length': 'int64'
}

# Path to the log file (ensure it exists or create it)
log_file_path = Path("/content/drive/MyDrive/AiLabData/notebook2_log.txt")
if not log_file_path.exists():
    with open(log_file_path, "w") as f:
        f.write("Notebook 2 Data Engineering Log\n")

# Check and handle 'priority_cat' column
target_column = 'priority_cat'

if target_column in dc.columns:
    print(f"\nVerificando la columna '{target_column}':")

    # Convert to numeric, coercing errors to NaN, then convert to nullable integer
    dc[target_column] = pd.to_numeric(dc[target_column], errors='coerce').astype('Int64')

    # Identify invalid values (NaN or not in [1, 2, 3])
    invalid_mask = dc[target_column].isnull() | ~dc[target_column].isin([1, 2, 3])
    invalid_count = invalid_mask.sum()
    total_rows = len(dc)
    invalid_percentage = (invalid_count / total_rows) * 100

    if invalid_count == 0:
        print(f"La columna '{target_column}' no tiene valores nulos o inválidos.")
        # Add 'priority_cat' to expected_columns with its detected dtype
        expected_columns[target_column] = str(dc[target_column].dtype)
        log_message = f"Columna '{target_column}' encontrada sin valores nulos o inválidos y añadida a columnas esperadas."
        print(log_message)
        with open(log_file_path, "a") as log_file:
            log_file.write(log_message + "\n")
    elif invalid_percentage <= 50:
        print(f"La columna '{target_column}' tiene {invalid_count} valores nulos o inválidos ({invalid_percentage:.2f}%). Eliminando filas con estos valores.")
        initial_rows = len(dc)
        dc.drop(dc[invalid_mask].index, inplace=True)
        rows_after_drop = len(dc)
        dropped_rows_count = initial_rows - rows_after_drop
        print(f"Se eliminaron {dropped_rows_count} filas debido a valores nulos o inválidos en '{target_column}'.")
        # Add 'priority_cat' to expected_columns after handling invalid data
        expected_columns[target_column] = str(dc[target_column].dtype)
        log_message = f"Columna '{target_column}' encontrada con {invalid_count} nulos o inválidos ({invalid_percentage:.2f}%). Se eliminaron {dropped_rows_count} filas."
        with open(log_file_path, "a") as log_file:
            log_file.write(log_message + "\n")
    else:
        print(f"La columna '{target_column}' tiene {invalid_count} valores nulos o inválidos ({invalid_percentage:.2f}%). Más del 50% son inválidos, eliminando la columna.")
        dc.drop(columns=[target_column], inplace=True)
        log_message = f"Columna '{target_column}' encontrada con {invalid_count} nulos o inválidos ({invalid_percentage:.2f}%). Más del 50% son inválidos, la columna ha sido eliminada."
        with open(log_file_path, "a") as log_file:
            log_file.write(log_message + "\n")
else:
    print(f"\nLa columna '{target_column}' no se encontró en el dataset.")
    log_message = f"Columna '{target_column}' no encontrada en el dataset."
    with open(log_file_path, "a") as log_file:
        log_file.write(log_message + "\n")


# Verificar si faltan columnas (based on potentially updated expected_columns)
missing_columns = [col for col in expected_columns.keys() if col not in dc.columns]

if missing_columns:
    # Si faltan columnas, imprimir mensaje de error y registrar en un log
    error_message = f"Lo siento, el dataset está incompleto. Faltan las siguientes columnas: {', '.join(missing_columns)}. No es posible continuar con el análisis."
    print(error_message)

    # Crear un archivo de log (append to existing log)
    with open(log_file_path, "a") as log_file:
        log_file.write("\nReporte de Columnas Faltantes:\n")
        for col in missing_columns:
            log_file.write(f"- {col}\n")
    print(f"Se ha actualizado el archivo de log en: {log_file_path}")

else:
    print("\nTodas las columnas esperadas están presentes.")
    log_message = "Todas las columnas esperadas están presentes."
    with open(log_file_path, "a") as log_file:
        log_file.write(log_message + "\n")


    # Eliminar las columnas que no están en la lista de columnas esperadas
    columns_to_drop = [col for col in dc.columns if col not in expected_columns.keys()]
    if columns_to_drop:
        print(f"\nEliminando columnas no esperadas: {', '.join(columns_to_drop)}")
        dc.drop(columns=columns_to_drop, inplace=True)
        log_message = f"Columnas no esperadas eliminadas: {', '.join(columns_to_drop)}"
        with open(log_file_path, "a") as log_file:
            log_file.write(log_message + "\n")
    else:
        print("\nNo se encontraron columnas no esperadas para eliminar.")
        log_message = "No se encontraron columnas no esperadas para eliminar."
        with open(log_file_path, "a") as log_file:
            log_file.write(log_message + "\n")


    print("\nColumnas restantes en el DataFrame después de eliminar las no esperadas:")
    print(dc.columns.tolist())

    # Mostrar información y primeras filas del DataFrame resultante
    print("\nInformación del DataFrame después de la validación y limpieza:")
    dc.info()
    print("\nPrimeras filas del DataFrame después de la validación y limpieza:")
    display(dc.head())


Verificando la columna 'priority_cat':
La columna 'priority_cat' no tiene valores nulos o inválidos.
Columna 'priority_cat' encontrada sin valores nulos o inválidos y añadida a columnas esperadas.

Todas las columnas esperadas están presentes.

Eliminando columnas no esperadas: ticket_id, day_of_week, day_of_week_num, company_id, company_size_cat, industry_cat, customer_tier_cat, org_users, region, region_cat, product_area_cat, booking_channel, booking_channel_cat, reported_by_role, reported_by_role_cat, customer_sentiment, customer_sentiment_cat, priority

Columnas restantes en el DataFrame después de eliminar las no esperadas:
['company_size', 'industry', 'customer_tier', 'past_30d_tickets', 'past_90d_incidents', 'product_area', 'customers_affected', 'error_rate_pct', 'downtime_min', 'payment_impact_flag', 'security_incident_flag', 'data_loss_flag', 'has_runbook', 'description_length', 'priority_cat']

Información del DataFrame después de la validación y limpieza:
<class 'pandas.cor

,company_size,industry,customer_tier,past_30d_tickets,past_90d_incidents,product_area,customers_affected,error_rate_pct,downtime_min,payment_impact_flag,security_incident_flag,data_loss_flag,has_runbook,description_length,priority_cat
0,Small,media,Basic,2,0,mobile,2,5.451201,6,0,0,0,0,227,1
1,Small,healthcare,Basic,2,3,analytics,24,2.495538,2,0,0,0,0,461,1
2,Small,gaming,Basic,3,1,notifications,1,3.328402,0,0,0,0,1,306,1
3,Small,media,Plus,2,2,analytics,25,2.931906,16,0,0,0,1,363,2
4,Small,ecommerce,Plus,5,1,analytics,28,3.255222,6,0,0,0,0,442,1


In [ ]:
#4 Estandarización del DataSet, asegurar que los features correspondan al tipo de dato esperado y manejo de nulos en numéricos

# Path to the log file
log_file_path = Path("/content/drive/MyDrive/AiLabData/notebook2_log.txt")

# Dictionary to store corrected values count per column
corrected_values_count = {}

print("\nVerificando y corrigiendo tipos de datos:")
# Checking and correcting data types:
for col, expected_dtype in expected_columns.items():
    if col in dc.columns:
        initial_dtype = dc[col].dtype
        initial_non_null_count = dc[col].notna().sum()
        unparsable_count = 0 # Inicializar unparsable_count a 0 al comienzo de cada iteración

        try:
            if expected_dtype == 'object':
                # If expected is object, ensure it's treated as object
                dc[col] = dc[col].astype(str)
            elif expected_dtype in ['int64', 'float64']:
                # Clean numerical columns: remove non-digit characters except dot for float
                if expected_dtype == 'int64':
                     dc[col] = dc[col].astype(str).str.replace(r'[^\d]', '', regex=True)
                elif expected_dtype == 'float64':
                     dc[col] = dc[col].astype(str).str.replace(r'[^\d.]', '', regex=True)

                # Convert to numeric, coercing errors. Invalid parsing will result in NaN.
                numeric_series = pd.to_numeric(dc[col], errors='coerce')

                # Count how many values were not originally NaN but became NaN after coercion
                # This indicates values that couldn't be parsed as numbers
                unparsable_count = dc[col].notna().sum() - numeric_series.notna().sum()

                # Fill NaN values (including those from coercion) with 0
                dc[col] = numeric_series.fillna(0)

                # Convert to the specific numeric type, handling nullable types
                if expected_dtype == 'int64':
                    dc[col] = dc[col].astype('Int64')
                elif expected_dtype == 'float64':
                    dc[col] = dc[col].astype('Float64')

            # Calculate the number of values that changed type or were filled with 0
            # This is a simpler way to estimate corrections after the fill/coerce steps
            # We can't easily track exact "corrections" per original value without more complex logic,
            # so we'll report on values that became 0 due to being unparsable.
            if unparsable_count > 0:
                 corrected_values_count[col] = unparsable_count


        except Exception as e:
            print(f"Error processing column {col}: {e}")


# Report on corrected values and write to log
print("\nReporte de corrección de tipos de datos y valores:")
# Report on data type and value correction:
if corrected_values_count:
    with open(log_file_path, "a") as log_file:
        log_file.write("\nReporte de Corrección de Tipos de Datos y Valores:\n")
        for col, count in corrected_values_count.items():
            message = f"- Columna '{col}': {count} valores que no pudieron ser convertidos a numérico se corrigieron a 0."
            print(message)
            log_file.write(message + "\n")
else:
    message = "No se encontraron valores no numéricos que requirieran corrección a 0 en las columnas numéricas esperadas."
    print(message)
    with open(log_file_path, "a") as log_file:
         log_file.write("\n" + message + "\n")


# Verify data types after correction
dtype_errors_after = []
for col, expected_dtype in expected_columns.items():
    if col in dc.columns:
        if dc[col].dtype.name != expected_dtype:
             # Handle Int64 vs int64 and Float64 vs float64
            if not ((expected_dtype == 'int64' and dc[col].dtype.name == 'Int64') or
                    (expected_dtype == 'float64' and dc[col].dtype.name == 'Float64')):
                dtype_errors_after.append(f"La columna '{col}' tiene el tipo de dato '{dc[col].dtype.name}' pero se esperaba '{expected_dtype}' después de la corrección.")

if dtype_errors_after:
    print("\nErrores en los tipos de datos después de la corrección:")
    # Print data type errors after correction if they exist:
    for error in dtype_errors_after:
        print(error)
        with open(log_file_path, "a") as log_file:
            log_file.write(error + "\n")
else:
    print("\nTodos los tipos de datos de las columnas esperadas coinciden con los definidos después de la corrección.")
    # All expected column data types match the defined ones after correction.


# Display info and head of the resulting DataFrame
# Mostrar información y primeras filas del DataFrame resultante
print("\nInformación del DataFrame después de la validación y corrección de tipos:")
dc.info()
print("\nPrimeras filas del DataFrame después de la validación y corrección de tipos:")
display(dc.head())


Verificando y corrigiendo tipos de datos:

Reporte de corrección de tipos de datos y valores:
No se encontraron valores no numéricos que requirieran corrección a 0 en las columnas numéricas esperadas.

Todos los tipos de datos de las columnas esperadas coinciden con los definidos después de la corrección.

Información del DataFrame después de la validación y corrección de tipos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company_size            50000 non-null  object 
 1   industry                50000 non-null  object 
 2   customer_tier           50000 non-null  object 
 3   past_30d_tickets        50000 non-null  Int64  
 4   past_90d_incidents      50000 non-null  Int64  
 5   product_area            50000 non-null  object 
 6   customers_affected      50000 non-null  Int64  
 7   error_rate_pct          50

,company_size,industry,customer_tier,past_30d_tickets,past_90d_incidents,product_area,customers_affected,error_rate_pct,downtime_min,payment_impact_flag,security_incident_flag,data_loss_flag,has_runbook,description_length,priority_cat
0,Small,media,Basic,2,0,mobile,2,5.451201,6,0,0,0,0,227,1
1,Small,healthcare,Basic,2,3,analytics,24,2.495538,2,0,0,0,0,461,1
2,Small,gaming,Basic,3,1,notifications,1,3.328402,0,0,0,0,1,306,1
3,Small,media,Plus,2,2,analytics,25,2.931906,16,0,0,0,1,363,2
4,Small,ecommerce,Plus,5,1,analytics,28,3.255222,6,0,0,0,0,442,1


In [ ]:
#5 Manejo de datos nulos en object

# Path to the log file
log_file_path = Path("/content/drive/MyDrive/AiLabData/notebook2_log.txt")

print("\nManejo de datos nulos:")
# Handling null values:

# Dictionary to store the number of null values found and corrected per column
null_correction_report = {}

# Iterate over columns to handle null values
for col in dc.columns:
    if dc[col].isnull().sum() > 0:
        # Count nulls before correction
        initial_null_count = dc[col].isnull().sum()

        if dc[col].dtype == 'object':
            # Replace nulls with 'UNKNOWN' for object type columns
            dc[col].fillna('UNKNOWN', inplace=True)
            corrected_count = initial_null_count - dc[col].isnull().sum()
            if corrected_count > 0:
                null_correction_report[col] = {'initial_nulls': initial_null_count, 'corrected_to': 'UNKNOWN', 'corrected_count': corrected_count}
        elif pd.api.types.is_numeric_dtype(dc[col]):
            # Replace nulls with 0 for numerical columns
            dc[col].fillna(0, inplace=True)
            corrected_count = initial_null_count - dc[col].isnull().sum()
            if corrected_count > 0:
                null_correction_report[col] = {'initial_nulls': initial_null_count, 'corrected_to': 0, 'corrected_count': corrected_count}

# Report on null value corrections and write to log
print("\nReporte de corrección de datos nulos:")
# Report on null value correction:
if null_correction_report:
    with open(log_file_path, "a") as log_file:
        log_file.write("\nReporte de Corrección de Datos Nulos:\n")
        for col, report_data in null_correction_report.items():
            message = (f"- Columna '{col}': Se encontraron {report_data['initial_nulls']} valores nulos. "
                       f"Se corrigieron {report_data['corrected_count']} valores a '{report_data['corrected_to']}'.")
            print(message)
            log_file.write(message + "\n")
else:
    message = "No se encontraron valores nulos que requirieran corrección."
    print(message)
    with open(log_file_path, "a") as log_file:
         log_file.write("\n" + message + "\n")

# Verify no more null values in the expected columns
print("\nVerificación de valores nulos después de la corrección:")
# Verification of null values after correction:
nulls_after_correction = dc.isnull().sum()
if nulls_after_correction.sum() == 0:
    print("No se encontraron valores nulos después de la corrección.")
else:
    print("Columnas con valores nulos después de la corrección:")
    display(nulls_after_correction[nulls_after_correction > 0])

# Display info and head of the resulting DataFrame
print("\nInformación del DataFrame después del manejo de datos nulos:")
dc.info()
print("\nPrimeras filas del DataFrame después del manejo de datos nulos:")
display(dc.head())


Manejo de datos nulos:

Reporte de corrección de datos nulos:
No se encontraron valores nulos que requirieran corrección.

Verificación de valores nulos después de la corrección:
No se encontraron valores nulos después de la corrección.

Información del DataFrame después del manejo de datos nulos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company_size            50000 non-null  object 
 1   industry                50000 non-null  object 
 2   customer_tier           50000 non-null  object 
 3   past_30d_tickets        50000 non-null  Int64  
 4   past_90d_incidents      50000 non-null  Int64  
 5   product_area            50000 non-null  object 
 6   customers_affected      50000 non-null  Int64  
 7   error_rate_pct          50000 non-null  Float64
 8   downtime_min            50000 non-null  Int64  
 9   pay

,company_size,industry,customer_tier,past_30d_tickets,past_90d_incidents,product_area,customers_affected,error_rate_pct,downtime_min,payment_impact_flag,security_incident_flag,data_loss_flag,has_runbook,description_length,priority_cat
0,Small,media,Basic,2,0,mobile,2,5.451201,6,0,0,0,0,227,1
1,Small,healthcare,Basic,2,3,analytics,24,2.495538,2,0,0,0,0,461,1
2,Small,gaming,Basic,3,1,notifications,1,3.328402,0,0,0,0,1,306,1
3,Small,media,Plus,2,2,analytics,25,2.931906,16,0,0,0,1,363,2
4,Small,ecommerce,Plus,5,1,analytics,28,3.255222,6,0,0,0,0,442,1


In [ ]:
# 6) Verificar y corregir columnas binarias

# Path to the log file
log_file_path = Path("/content/drive/MyDrive/AiLabData/notebook2_log.txt")

# Define binary columns based on user's specified list
binary_cols = ['downtime_min', 'payment_impact_flag', 'security_incident_flag', 'data_loss_flag', 'has_runbook']

print("\nVerificando y corrigiendo columnas binarias:")
# Checking and correcting binary columns:

binary_correction_report = {}

for col in binary_cols:
    if col in dc.columns:
        # Ensure the column is numeric before checking values
        if pd.api.types.is_numeric_dtype(dc[col]):
            # Identify values that are not 0 or 1 and are not null
            invalid_binary_mask = ~dc[col].isin([0, 1]) & dc[col].notna()
            invalid_binary_count = invalid_binary_mask.sum()

            if invalid_binary_count > 0:
                # Force invalid numeric values to 0
                dc.loc[invalid_binary_mask, col] = 0
                total_rows = len(dc)
                corrected_percentage = (invalid_binary_count / total_rows) * 100
                binary_correction_report[col] = {'count': invalid_binary_count, 'percentage': corrected_percentage}
                print(f"- Columna '{col}': Se encontraron y corrigieron {invalid_binary_count} valores no binarios ({corrected_percentage:.2f}%) a 0.")
            else:
                print(f"- Columna '{col}': No se encontraron valores no binarios.")
        else:
            print(f"- Advertencia: La columna '{col}' existe pero no es de tipo numérico. No se verificaron los valores binarios.")
    else:
        print(f"- Advertencia: La columna '{col}' no se encontró en el DataFrame.")

# Report on binary value corrections and write to log
print("\nReporte de corrección de valores binarios:")
# Report on binary value correction:
if binary_correction_report:
    with open(log_file_path, "a") as log_file:
        log_file.write("\nReporte de Corrección de Valores Binarios:\n")
        for col, report_data in binary_correction_report.items():
            message = (f"- Columna '{col}': Se corrigieron {report_data['count']} valores no binarios "
                       f"({report_data['percentage']:.2f}%) a 0.")
            print(message)
            log_file.write(message + "\n")
else:
    message = "No se encontraron valores no binarios que requirieran corrección en las columnas binarias esperadas."
    print(message)
    with open(log_file_path, "a") as log_file:
         log_file.write("\n" + message + "\n")

# Display info and head of the resulting DataFrame
print("\nInformación del DataFrame después de la verificación y corrección de binarios:")
dc.info()
print("\nPrimeras filas del DataFrame después de la verificación y corrección de binarios:")
display(dc.head())


Verificando y corrigiendo columnas binarias:
- Columna 'downtime_min': Se encontraron y corrigieron 24471 valores no binarios (48.94%) a 0.
- Columna 'payment_impact_flag': No se encontraron valores no binarios.
- Columna 'security_incident_flag': No se encontraron valores no binarios.
- Columna 'data_loss_flag': No se encontraron valores no binarios.
- Columna 'has_runbook': No se encontraron valores no binarios.

Reporte de corrección de valores binarios:
- Columna 'downtime_min': Se corrigieron 24471 valores no binarios (48.94%) a 0.

Información del DataFrame después de la verificación y corrección de binarios:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company_size            50000 non-null  object 
 1   industry                50000 non-null  object 
 2   customer_tier           50000 non-null  object 
 3

,company_size,industry,customer_tier,past_30d_tickets,past_90d_incidents,product_area,customers_affected,error_rate_pct,downtime_min,payment_impact_flag,security_incident_flag,data_loss_flag,has_runbook,description_length,priority_cat
0,Small,media,Basic,2,0,mobile,2,5.451201,0,0,0,0,0,227,1
1,Small,healthcare,Basic,2,3,analytics,24,2.495538,0,0,0,0,0,461,1
2,Small,gaming,Basic,3,1,notifications,1,3.328402,0,0,0,0,1,306,1
3,Small,media,Plus,2,2,analytics,25,2.931906,0,0,0,0,1,363,2
4,Small,ecommerce,Plus,5,1,analytics,28,3.255222,0,0,0,0,0,442,1


In [ ]:
#7 Aplicamos log1p(x) a 'customers_affected', error_rate_pct, and 'downtime_min'
# Applying log1p(x) to 'customers_affected', error_rate_pct, and 'downtime_min'
cols_to_log = ['customers_affected', 'downtime_min', 'error_rate_pct']

for col in cols_to_log:
    new_col_name = f'{col}_log'
    # Ensure column exists and is numeric before applying log1p
    if col in dc.columns and pd.api.types.is_numeric_dtype(dc[col]):
        dc[new_col_name] = np.log1p(dc[col])
        print(f"Log1p transformación aplicada a '{col}', resultado guardado en '{new_col_name}'.")
    else:
        print(f"Advertencia: La columna '{col}' no se encontró o no es numérica. No se aplicó log1p.")


# Eliminar las columnas originales después de la transformación
# Drop the original columns after transformation
dc.drop(columns=cols_to_log, inplace=True, errors='ignore')
print(f"\nColumnas originales eliminadas: {cols_to_log}")


print("\nPrimeras 5 filas con las nuevas columnas log:")
# Display first 5 rows with the new log columns
log_cols_created = [f'{col}_log' for col in cols_to_log if f'{col}_log' in dc.columns]
if log_cols_created:
    display(dc[log_cols_created].head())
else:
    print("No se crearon columnas log.")

print("\nValores máximos y mínimos de las columnas log transformadas:")
# Display max and min values of the log-transformed columns
if log_cols_created:
    display(dc[log_cols_created].agg(['min', 'max']))
else:
     print("No hay columnas log para mostrar min/max.")

# Mostrar información del DataFrame para verificar las columnas
# Display DataFrame info to verify columns
print("\nInformación del DataFrame después de la transformación y eliminación:")
dc.info()

Log1p transformación aplicada a 'customers_affected', resultado guardado en 'customers_affected_log'.
Log1p transformación aplicada a 'downtime_min', resultado guardado en 'downtime_min_log'.
Log1p transformación aplicada a 'error_rate_pct', resultado guardado en 'error_rate_pct_log'.

Columnas originales eliminadas: ['customers_affected', 'downtime_min', 'error_rate_pct']

Primeras 5 filas con las nuevas columnas log:


,customers_affected_log,downtime_min_log,error_rate_pct_log
0,1.098612,0.0,1.864266
1,3.218876,0.0,1.251487
2,0.693147,0.0,1.465198
3,3.258097,0.0,1.369124
4,3.367296,0.0,1.448147



Valores máximos y mínimos de las columnas log transformadas:


,customers_affected_log,downtime_min_log,error_rate_pct_log
min,0.000000,0.000000,0.000000
max,8.658345,0.693147,4.392157



Información del DataFrame después de la transformación y eliminación:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company_size            50000 non-null  object 
 1   industry                50000 non-null  object 
 2   customer_tier           50000 non-null  object 
 3   past_30d_tickets        50000 non-null  Int64  
 4   past_90d_incidents      50000 non-null  Int64  
 5   product_area            50000 non-null  object 
 6   payment_impact_flag     50000 non-null  Int64  
 7   security_incident_flag  50000 non-null  Int64  
 8   data_loss_flag          50000 non-null  Int64  
 9   has_runbook             50000 non-null  Int64  
 10  description_length      50000 non-null  Int64  
 11  priority_cat            50000 non-null  Int64  
 12  customers_affected_log  50000 non-null  Float64
 13  downtime_min_log    

In [ ]:
#8 aplicmo z-score a variabls numericas
from sklearn.preprocessing import StandardScaler

cols_to_scale = [
    "past_30d_tickets",
    "past_90d_incidents",
    "description_length",
    "downtime_min_log",
    "customers_affected_log",
    "error_rate_pct_log"
]

# Iniciamos StandardScaler
scaler = StandardScaler()

 #Aplicamos la escalación estsndard (z-score)
dc[cols_to_scale] = scaler.fit_transform(dc[cols_to_scale])

print("Datos escalados (primeras 5 filas):")
display(dc[cols_to_scale].head())

print("\nValores Min and Max después de escalar:")
display(dc[cols_to_scale].agg(['min', 'max']))

Datos escalados (primeras 5 filas):


,past_30d_tickets,past_90d_incidents,description_length,downtime_min_log,customers_affected_log,error_rate_pct_log
0,-0.652358,-1.257509,-1.368881,-0.084921,-1.387750,0.553666
1,-0.652358,0.786423,0.874317,-0.084921,-0.228376,-0.302346
2,-0.293185,-0.576198,-0.611562,-0.084921,-1.609461,-0.003806
3,-0.652358,0.105113,-0.065142,-0.084921,-0.206930,-0.138015
4,0.425160,-0.576198,0.692177,-0.084921,-0.147219,-0.027626



Valores Min and Max después de escalar:


,past_30d_tickets,past_90d_incidents,description_length,downtime_min_log,customers_affected_log,error_rate_pct_log
min,-1.370703,-1.257509,-3.353248,-0.084921,-1.988478,-2.050592
max,5.453574,6.918218,4.267873,11.775602,2.745960,4.084965


In [ ]:
#9  Define the allowed categories for each column (re-defining here for clarity in this cell)
# Definir las categorías permitidas para cada columna (re-definidas aquí para mayor claridad en esta celda)
allowed_categories = {
    'company_size': ['Small', 'Medium', 'Large'],
    'industry': ['media', 'healthcare', 'gaming', 'ecommerce', 'saas_b2b', 'logistics', 'fintech'],
    'customer_tier': ['Basic', 'Plus', 'Enterprise'],
    'product_area': ['mobile', 'analytics', 'notifications', 'auth', 'billing', 'data_pipeline']
}

# Identify categorical columns in the DataFrame 'dc' that should be encoded
# Identificar las columnas categóricas en el DataFrame 'dc' que deben ser codificadas
cols_to_encode = [col for col in ['company_size', 'industry', 'customer_tier', 'product_area'] if col in dc.columns and dc[col].dtype == 'object']

print("Aplicando One-Hot Encoding a las columnas categóricas:")
# Applying One-Hot Encoding to categorical columns:

# First, handle unexpected categories by grouping them into 'other'
# Primero, manejar las categorías no esperadas agrupándolas en 'other'
print("Manejo de subcategorías no esperadas antes de One-Hot Encoding:")
for col in cols_to_encode:
    if col in dc.columns and dc[col].dtype == 'object':
        # Identify unexpected values
        unexpected_values = dc[col][~dc[col].isin(allowed_categories.get(col, []))].unique()

        if len(unexpected_values) > 0:
            # Replace unexpected values with 'other'
            dc[col] = dc[col].apply(lambda x: x if x in allowed_categories.get(col, []) else 'other')
            print(f"- Columna '{col}': Se agruparon {len(unexpected_values)} subcategorías no esperadas en 'other'.")
        else:
            print(f"- Columna '{col}': No se encontraron subcategorías no esperadas.")
    elif col in dc.columns:
         print(f"- Advertencia: La columna '{col}' existe pero no es de tipo 'object'. No se aplicó manejo de subcategorías.")
    else:
         print(f"- Advertencia: La columna '{col}' no se encontró en el DataFrame.")


# Now, apply one-hot encoding to the (potentially modified) columns
# Ahora, aplicar one-hot encoding a las columnas (potencialmente modificadas)

# Use pd.get_dummies to apply one-hot encoding, including the 'other' category if created
# Usar pd.get_dummies para aplicar one-hot encoding, incluyendo la categoría 'other' si fue creada
dc = pd.get_dummies(dc, columns=cols_to_encode, drop_first=False, dtype=int) # Keep all categories for now


print("\nDataFrame después de One-Hot Encoding:")
# DataFrame after One-Hot Encoding:
dc.info()
display(dc.head())

Aplicando One-Hot Encoding a las columnas categóricas:
Manejo de subcategorías no esperadas antes de One-Hot Encoding:
- Columna 'company_size': No se encontraron subcategorías no esperadas.
- Columna 'industry': No se encontraron subcategorías no esperadas.
- Columna 'customer_tier': No se encontraron subcategorías no esperadas.
- Columna 'product_area': No se encontraron subcategorías no esperadas.

DataFrame después de One-Hot Encoding:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   past_30d_tickets            50000 non-null  float64
 1   past_90d_incidents          50000 non-null  float64
 2   payment_impact_flag         50000 non-null  Int64  
 3   security_incident_flag      50000 non-null  Int64  
 4   data_loss_flag              50000 non-null  Int64  
 5   has_runbook                 50000 non-null 

,past_30d_tickets,past_90d_incidents,payment_impact_flag,security_incident_flag,data_loss_flag,has_runbook,description_length,priority_cat,customers_affected_log,downtime_min_log,...,industry_saas_b2b,customer_tier_Basic,customer_tier_Enterprise,customer_tier_Plus,product_area_analytics,product_area_auth,product_area_billing,product_area_data_pipeline,product_area_mobile,product_area_notifications
0,-0.652358,-1.257509,0,0,0,0,-1.368881,1,-1.387750,-0.084921,...,0,1,0,0,0,0,0,0,1,0
1,-0.652358,0.786423,0,0,0,0,0.874317,1,-0.228376,-0.084921,...,0,1,0,0,1,0,0,0,0,0
2,-0.293185,-0.576198,0,0,0,1,-0.611562,1,-1.609461,-0.084921,...,0,1,0,0,0,0,0,0,0,1
3,-0.652358,0.105113,0,0,0,1,-0.065142,2,-0.206930,-0.084921,...,0,0,0,1,1,0,0,0,0,0
4,0.425160,-0.576198,0,0,0,0,0.692177,1,-0.147219,-0.084921,...,0,0,0,1,1,0,0,0,0,0


# **BALANCEO DE CLASE TARGET: priority_cat**

In [ ]:

#10 balanceamos priority con class_weight
from sklearn.utils import class_weight

# Calculate class weights
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(dc['priority_cat'].astype(str)), # Convert unique classes to numpy array of strings
    y=dc['priority_cat'].astype(str) # Also convert y to strings to match classes
)

# Create a dictionary of class weights
class_weight_dict = {
    cls: weight for cls, weight in zip(np.unique(dc['priority_cat'].astype(str)), class_weights) # Use unique classes as strings for dictionary keys
}

print("Class weights for 'priority':")
display(class_weight_dict)

Class weights for 'priority':


{'1': np.float64(0.6666666666666666),
 '2': np.float64(0.9523809523809523),
 '3': np.float64(2.2222222222222223)}

In [ ]:
# 11) Guarda dc en la misma carpeta de donde tomaste Support_tickets, con el nombre Support_tickets_training_set en formato csv

# Get the directory of the original data file
data_directory = Path(DATA_PATH).parent

# Define the path for the new CSV file
output_file_path = data_directory / "Support_tickets_training_set.csv"

# Save the DataFrame to CSV
dc.to_csv(output_file_path, index=False)

print(f"DataFrame guardado exitosamente en: {output_file_path}")

DataFrame guardado exitosamente en: /content/drive/MyDrive/AiLabData/Support_tickets_training_set.csv
